In [1]:
# A notebook to test the ability of a large language model to use a bar's inventory for various tasks
# We will use the langchain library to create agents that can interact with the inventory to allow for contextual
# conversations

# Initial Imports
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import openai

# Load in the environment variables
load_dotenv()


True

In [2]:
# Get the OpenAI API key and org key from the .env file
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG")

In [3]:
# Create a dataframe of the inventory

# A list of 50 common spirits
spirits = ['Vodka', 'Gin', 'Rum', 'Tequila', 'Whiskey', 'Brandy', 'Scotch', 'Bourbon', 'Cognac', 'Absinthe', 
           'Sake', 'Rye', 'Mezcal', 'Amaretto', 'Baileys', 'Campari', 'Cointreau', 'Frangelico', 'Grand Marnier', 
           'Jägermeister', 'Kahlúa', 'Limoncello', 'Sambuca', 'Southern Comfort', 'Tia Maria', 'Triple Sec', 
           'Pisco', 'Port', 'Sherry', 'Vermouth', 'Aperol', 'Bitters', 'Chartreuse', 'Chambord', 'Fernet Branca', 
           'Galliano', 'Goldschlager', 'Malibu', 'Midori', 'Peach Schnapps', 'Peppermint Schnapps', 'Sloe Gin', 
           'St Germain', 'Umeshu', 'Anisette', 'Aquavit', 'Becherovka', 'Bénédictine', 'Crème de Cacao', 'Crème de Menthe']

# Item IDs
item_ids = np.arange(1, 51)

# Randomly generated quantities, volumes, and costs for each spirit
quantities = np.random.randint(1, 10, size=50)
volumes = np.random.choice([750, 1000], size=50)
costs = np.random.uniform(20, 100, size=50)

# Calculate cost per ml
costs_per_ml = costs / volumes

# Create the DataFrame
df = pd.DataFrame({
    'ItemID': item_ids,
    'Name': spirits,
    'Type': 'Spirit',
    'Quantity': quantities,
    'Volume per Unit (ml)': volumes,
    'Cost per Unit': costs,
    'Cost per ml': costs_per_ml
})

print(df)


    ItemID                 Name    Type  Quantity  Volume per Unit (ml)  \
0        1                Vodka  Spirit         6                   750   
1        2                  Gin  Spirit         2                  1000   
2        3                  Rum  Spirit         1                  1000   
3        4              Tequila  Spirit         9                   750   
4        5              Whiskey  Spirit         5                   750   
5        6               Brandy  Spirit         6                   750   
6        7               Scotch  Spirit         1                   750   
7        8              Bourbon  Spirit         1                   750   
8        9               Cognac  Spirit         6                   750   
9       10             Absinthe  Spirit         2                  1000   
10      11                 Sake  Spirit         5                   750   
11      12                  Rye  Spirit         1                   750   
12      13               

In [4]:
# Make Item ID the index
df.set_index('ItemID', inplace=True)

In [5]:
# Save the DataFrame to a csv file
df.to_csv('inventory.csv')

In [6]:
# Add the wolfram appid to the environment
os.environ['WOLFRAM_ALPHA_APPID'] = os.getenv('WOLFRAM_APP_ID')

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, load_tools, create_pandas_dataframe_agent, Tool
from langchain.memory import ConversationBufferMemory

chat_agent = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature = 0, verbose=True, openai_api_key=openai.api_key, openai_organization=openai.organization)

# Create a csv agent that can access our inventory via the dataframe
inventory_agent = create_pandas_dataframe_agent(llm = chat_agent, df = df, verbose=True)

# Import tools
tools = load_tools(llm = chat_agent, tool_names=['python_repl', 'wolfram-alpha', 'llm-math'])


# Apend the inventory agent to the tools
tools.append(
    Tool.from_function(
        func = inventory_agent.run,
        name = "Liquor Inventory Queries",
        description = "Queries the liquor inventory for information about a specific item",
    )
)

tool_names = [tool.name for tool in tools]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages = True)
agent_executor = initialize_agent(agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, tools=tools, verbose=True, llm = chat_agent, memory = memory)


In [12]:
agent_executor.run("Create a craft cocktail that has a holiday theme that uses at least 1 spirit from the inventory.  Calculate the cost per drink of any\
                   liquors from the inventory that you use in the recipe.  Please give the recipe a creative name and a description of the drink.")




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Liquor Inventory Queries",
  "action_input": "gin, rum, vodka, tequila, whiskey"
}
```



> Entering new AgentExecutor chain...
Thought: We need to extract the names of the spirits from the dataframe
Action: Use pandas' `unique()` method to get the unique values in the 'Name' column of the dataframe
Action Input: `df['Name'].unique()`
Observation: Use pandas' `unique()` method to get the unique values in the 'Name' column of the dataframe is not a valid tool, try another one.
Thought:We can use pandas' `value_counts()` method to get the count of each unique value in the 'Name' column of the dataframe, and then extract the index of the resulting series to get the unique names of the spirits.
Action: Use pandas' `value_counts()` method to get the count of each unique value in the 'Name' column of the dataframe, and then extract the index of the resulting series to get the unique names of the spirits.
Action Input: `df['Na